In [15]:
import warnings
import pandas as pd
import numpy as np
import os
import sys # error msg
import operator # sorting
from math import *

from read_trace import *
from avgblkmodel import *

warnings.filterwarnings("ignore", category=np.VisibleDeprecationWarning)

# gpu info

In [16]:
gtx950 = DeviceInfo()
gtx950.sm_num = 6
gtx950.sharedmem_per_sm = 49152
gtx950.reg_per_sm = 65536
gtx950.maxthreads_per_sm = 2048

# single stream info

In [17]:
data_size = 23000
trace_file = './1cke/trace_' + str(data_size) + '.csv'
df_trace = trace2dataframe(trace_file) # read the trace to the dataframe

In [18]:
df_trace

,Start,Duration,Grid X,Grid Y,Grid Z,Block X,Block Y,Block Z,Registers Per Thread,Static SMem,Dynamic SMem,Size,Throughput,Device,Context,Stream,Name
0,ms,us,NaN,NaN,NaN,NaN,NaN,NaN,NaN,B,B,KB,GB/s,NaN,NaN,NaN,NaN
1,526.961828,16.672000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,89.843750,5.139256,GeForce GTX 950 (0),1.0,13.0,[CUDA memcpy HtoD]
2,526.979716,16.224000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,89.843750,5.281168,GeForce GTX 950 (0),1.0,13.0,[CUDA memcpy HtoD]
3,527.157829,61.056000,90.0,1.0,1.0,256.0,1.0,1.0,28.0,0,0,NaN,NaN,GeForce GTX 950 (0),1.0,13.0,"kernel_vectorAdd(float const *, float const *,..."
4,527.221349,15.904000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,89.843750,5.387429,GeForce GTX 950 (0),1.0,13.0,[CUDA memcpy DtoH]


In [19]:
df_single_stream = model_param_from_trace_v1(df_trace)
df_single_stream.head(20)

,api_type,start,end,duration
0,h2d,526.961828,526.978500,0.016672
1,h2d,526.979716,526.995940,0.016224
2,kern,527.157829,527.218885,0.061056
3,d2h,527.221349,527.237253,0.015904


In [20]:
df_s1 = reset_starting(df_single_stream)

In [21]:
df_s1

,api_type,start,end,duration
0,h2d,0.000000,0.016672,0.016672
1,h2d,0.017888,0.034112,0.016224
2,kern,0.196001,0.257057,0.061056
3,d2h,0.259521,0.275425,0.015904


### running 2cke case

In [22]:
stream_num = 2

df_cke_list = []
for x in range(stream_num):
    df_cke_list.append(df_s1.copy(deep=True))

In [23]:
df_cke_list[0]

,api_type,start,end,duration
0,h2d,0.000000,0.016672,0.016672
1,h2d,0.017888,0.034112,0.016224
2,kern,0.196001,0.257057,0.061056
3,d2h,0.259521,0.275425,0.015904


In [24]:
df_cke_list[1]

,api_type,start,end,duration
0,h2d,0.000000,0.016672,0.016672
1,h2d,0.017888,0.034112,0.016224
2,kern,0.196001,0.257057,0.061056
3,d2h,0.259521,0.275425,0.015904


In [25]:
H2D_H2D_OVLP_TH = 3.158431

for i in range(1,stream_num):
    # compute the time for the init data transfer
    stream_startTime = find_whentostart_comingStream(df_cke_list[i-1], H2D_H2D_OVLP_TH)
    print('stream_startTime : {}'.format(stream_startTime))
    df_cke_list[i].start += stream_startTime
    df_cke_list[i].end   += stream_startTime

stream_startTime : 0.0341120000001


In [26]:
df_cke_list[0]

,api_type,start,end,duration
0,h2d,0.000000,0.016672,0.016672
1,h2d,0.017888,0.034112,0.016224
2,kern,0.196001,0.257057,0.061056
3,d2h,0.259521,0.275425,0.015904


In [27]:
df_cke_list[1]

,api_type,start,end,duration
0,h2d,0.034112,0.050784,0.016672
1,h2d,0.052000,0.068224,0.016224
2,kern,0.230113,0.291169,0.061056
3,d2h,0.293633,0.309537,0.015904
